In [10]:
import numpy as np
import pandas as pd
from small_text import LeastConfidence, PoolBasedActiveLearner, random_initialization_balanced, QueryStrategy, TransformersDataset
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from synergy_dataset import Dataset, iter_datasets

from small_text.classifiers.factories import SklearnClassifierFactory
from sklearn.naive_bayes import MultinomialNB
from small_text.data.datasets import SklearnDataset
from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.over_sampling import SMOTEN

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer

In [ ]:
def query_cosine_similarity(QueryStrategy):
    def query(self, clf, dataset, indices_unlabeled, indices_labeled, y, n=10):
        results = []
        for index in indices_unlabeled:
            similarity = util.cos_sim(dataset[indices_labeled], dataset[index])
            results.append((index, similarity.item()))
        results.sort(key=lambda x: x[1], reverse=True)
        return [index for index, similarity in results[:n]]

In [2]:
def cria_dataset(dataset):
    ds = Dataset(dataset)
    ds = ds.to_frame()
    ds = ds.fillna('')
    title = ds['title']
    abstract = ds['abstract']
    X = np.array([x[0] + ' ' + x[1] for x in zip(title, abstract)])
    y = np.array(ds['label_included'])

    return X, y

In [11]:
dataset = 'Muthu_2021'
model_checkpoint = 'all-distilroberta-v1'
transformer_model = 'allenai/specter2_base'
model = SentenceTransformer(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(transformer_model)

In [12]:
X, y = cria_dataset(dataset)
ds = TransformersDataset.from_arrays(X, y, tokenizer, target_labels=np.array([0, 1]), max_length=128)

/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '


AttributeError: 'list' object has no attribute 'shape'